In [1]:
!pip install -e ../TrackSelectorDNN/ &> /tmp/install.log 

In [2]:
from cluster.RayJobManager import RayJobManager

In [11]:
RAY_ADDRESS  = "http://10.100.192.230:8265"
#RAY_ADDRESS  = "local"
ENTRYPOINT   = "python3 cluster/tune_grid.py"
WORKING_DIR  = "/eos/user/e/ecoradin/GitHub/TrackSelectorDNN-Ray/"
PIP = {
    "packages": [
        "torch",
        "ray[tune]",
        "git+https://github.com/EmanueleCoradin/TrackSelectorDNN.git@main"
    ],
    "pip_check": False,
    "pip_install_options": ["--verbose"]
}

ray_manager = RayJobManager(ray_address=RAY_ADDRESS)

In [12]:
job_id = ray_manager.submit_job(
    entrypoint=ENTRYPOINT,
    runtime_env={
        "working_dir": WORKING_DIR, 
        "pip": PIP, 
    }
    
)

ray_manager.wait_until_done(job_id)

2025-11-14 19:09:03,621	INFO dashboard_sdk.py:338 -- Uploading package gcs://_ray_pkg_686c4a80415d7a47.zip.
2025-11-14 19:09:03,623	INFO packaging.py:588 -- Creating a file package for local module '/eos/user/e/ecoradin/GitHub/TrackSelectorDNN-Ray/'.


Submitted Ray job: raysubmit_WHd15USSZJYJnf9h
Job status: PENDING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: SUCCEEDED

--- Job Logs ---
2025-11-14 11:09:04,693	INFO job_manager.py:568 -- Runtime env is setting up.
/home/ray/anaconda3/lib/python3.9/site-packages/ray/tune/impl/tuner_internal.py:144: RayDeprecationWarning: The `RunConfig` class should be imported from `ray.tune` when passing it to the Tuner. Please update your imports. See this issue for more context and migration options: https://github.com/ray-project/ray/issues/49454. Disable these warnings by set

In [2]:
#Uncomment to run locally

from ray import tune
from TrackSelectorDNN.configs.schema import load_config
from TrackSelectorDNN.tune.trainable import trainable

base_cfg = load_config("base.yaml").model_dump()

search_space = {
    **base_cfg,  # include all validated defaults
    "lambda_sym": 0.1,
    "lr": tune.loguniform(1e-5, 1e-2),
    "weight_decay": tune.loguniform(1e-5, 1e-2),
    "latent_dim": tune.choice([32, 64]),
    "netA_hidden_dim": tune.choice([32, 64]),
    "netB_hidden_dim": tune.choice([32, 64]),
    "pooling_type": tune.choice(["sum", "mean"]),
    "netA_hidden_layers": tune.choice([1, 2]),
    "netB_hidden_layers": tune.choice([1, 2]),
}


tuner = tune.Tuner(
    tune.with_resources(
        trainable,
        {"cpu": 1, "gpu": .25}  # adjust CPU/GPU numbers as needed
    ),
    param_space=search_space,
    tune_config=tune.TuneConfig(metric="val_loss", mode="min", num_samples=4),
)

results = tuner.fit()


(trainable pid=15876) [Epoch 1/5] train_loss=0.6267, val_loss=0.5771, val_acc=0.7115


(trainable pid=15876) /eos/user/e/ecoradin/.local/lib/python3.11/site-packages/ray/train/_internal/session.py:792: RayDeprecationWarning: `ray.train.report` should be switched to `ray.tune.report` when running in a function passed to Ray Tune. This will be an error in the future. See this issue for more context: https://github.com/ray-project/ray/issues/49454
(trainable pid=15876)   _log_deprecation_warning(
(trainable pid=15876) /eos/user/e/ecoradin/.local/lib/python3.11/site-packages/ray/tune/trainable/trainable_fn_utils.py:41: RayDeprecationWarning: The `Checkpoint` class should be imported from `ray.tune` when passing it to `ray.tune.report` in a Tune function. Please update your imports. See this issue for more context and migration options: https://github.com/ray-project/ray/issues/49454. Disable these warnings by setting the environment variable: RAY_TRAIN_ENABLE_V2_MIGRATION_WARNINGS=0
(trainable pid=15876)   _log_deprecation_warning(
(trainable pid=15876) Checkpoint successful

(trainable pid=15876) [Epoch 2/5] train_loss=0.5512, val_loss=0.5239, val_acc=0.7288 [repeated 4x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)


(trainable pid=15875) /eos/user/e/ecoradin/.local/lib/python3.11/site-packages/ray/train/_internal/session.py:792: RayDeprecationWarning: `ray.train.report` should be switched to `ray.tune.report` when running in a function passed to Ray Tune. This will be an error in the future. See this issue for more context: https://github.com/ray-project/ray/issues/49454 [repeated 3x across cluster]
(trainable pid=15875)   _log_deprecation_warning( [repeated 6x across cluster]
(trainable pid=15875) /eos/user/e/ecoradin/.local/lib/python3.11/site-packages/ray/tune/trainable/trainable_fn_utils.py:41: RayDeprecationWarning: The `Checkpoint` class should be imported from `ray.tune` when passing it to `ray.tune.report` in a Tune function. Please update your imports. See this issue for more context and migration options: https://github.com/ray-project/ray/issues/49454. Disable these warnings by setting the environment variable: RAY_TRAIN_ENABLE_V2_MIGRATION_WARNINGS=0 [repeated 3x across cluster]
(train

(trainable pid=15876) [Epoch 3/5] train_loss=0.5142, val_loss=0.4977, val_acc=0.7194 [repeated 4x across cluster]


...

(trainable pid=15876) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/eos/user/e/ecoradin/ray_results/trainable_2025-11-19_20-47-17/trainable_fb2cf_00002_2_latent_dim=32,lr=0.0000,netA_hidden_dim=64,netA_hidden_layers=1,netB_hidden_dim=32,netB_hidden_layers=1,po_2025-11-19_20-47-37/checkpoint_000004)
(trainable pid=15876) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/eos/user/e/ecoradin/ray_results/trainable_2025-11-19_20-47-17/trainable_fb2cf_00002_2_latent_dim=32,lr=0.0000,netA_hidden_dim=64,netA_hidden_layers=1,netB_hidden_dim=32,netB_hidden_layers=1,po_2025-11-19_20-47-37/checkpoint_000005)


(trainable pid=15877) [Epoch 5/5] train_loss=0.2866, val_loss=0.2744, val_acc=0.8882
(trainable pid=15877) [DEBUG] Final report to Ray: /eos/user/e/ecoradin/GitHub/TrackSelectorDNN/runs/fb2cf_00000


(trainable pid=15877) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/eos/user/e/ecoradin/ray_results/trainable_2025-11-19_20-47-17/trainable_fb2cf_00000_0_latent_dim=32,lr=0.0032,netA_hidden_dim=64,netA_hidden_layers=2,netB_hidden_dim=64,netB_hidden_layers=1,po_2025-11-19_20-47-37/checkpoint_000004) [repeated 2x across cluster]


(trainable pid=15875) [Epoch 5/5] train_loss=0.2885, val_loss=0.2829, val_acc=0.8879


2025-11-19 20:49:16,898	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/eos/user/e/ecoradin/ray_results/trainable_2025-11-19_20-47-17' in 0.0729s.
2025-11-19 20:49:16,916	INFO tune.py:1041 -- Total run time: 100.10 seconds (99.52 seconds for the tuning loop).


In [3]:
best_result = results.get_best_result(metric="val_loss", mode="min")

print("Best config found:")
print(best_result.config)

best_ckpt_dir = best_result.checkpoint
print("Best checkpoint path:", best_ckpt_dir)

Best config found:
{'hit_input_dim': 9, 'track_feat_dim': 25, 'latent_dim': 256, 'pooling_type': 'softmax', 'netA_hidden_dim': 64, 'netA_hidden_layers': 3, 'netA_batchnorm': True, 'netA_activation': 'silu', 'netB_hidden_dim': 128, 'netB_hidden_layers': 6, 'netB_batchnorm': True, 'netB_activation': 'silu', 'lr': 0.001018182389130653, 'epochs': 5, 'batch_size': 128, 'patience': 15, 'delta': 0.0, 'base_checkpoint_directory': '/eos/user/e/ecoradin/GitHub/TrackSelectorDNN/runs', 'dataset_type': 'production', 'dummy_load_path': None, 'train_path': '/eos/user/e/ecoradin/GitHub/TrackSelectorDNN/TrackSelectorDNN/data/tracks_train_dataset.pt', 'val_path': '/eos/user/e/ecoradin/GitHub/TrackSelectorDNN/TrackSelectorDNN/data/tracks_val_dataset.pt', 'test_path': '/eos/user/e/ecoradin/GitHub/TrackSelectorDNN/TrackSelectorDNN/data/tracks_test_dataset.pt', 'max_hits': 16, 'weight_decay': 6.255132107953883e-05}
Best checkpoint path: Checkpoint(filesystem=local, path=/eos/user/e/ecoradin/ray_results/trai